In [ ]:
import pandas as pd
import warnings 
warnings.simplefilter('ignore')
from sklearn.ensemble import RandomForestClassifier

In [39]:
train=pd.read_csv('train.csv', index_col=0)
xTest=pd.read_csv('test.csv', index_col=0)

In [40]:
train.shape, xTest.shape

((891, 11), (418, 10))

In [41]:
train.columns, xTest.columns

(Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
        'Fare', 'Cabin', 'Embarked'],
       dtype='object'),
 Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
        'Cabin', 'Embarked'],
       dtype='object'))

In [42]:
yTrain=train['Survived']
xTrain=train.iloc[:, 1:]

In [43]:
xTrain.shape, yTrain.shape

((891, 10), (891,))

In [44]:
def xCleaner(X):
    '''clean up the x df for exploratory analysis and model building''' 
    X['Name']=X['Name'].str.split(pat=', |\.') 
    for i in range(len(X)):
        X['Name'].iloc[i]=X['Name'].iloc[i][1]
    X['Name']=X.Name.apply(str)

    X['Age'].fillna((X['Age'].mean()), inplace=True)
    bins=pd.cut(X['Age'], bins=[1,6,13,20,51,100],
                labels=['toddler', 'adolescent', 'teen', 'adult', 'elderly']) 
    bin2=pd.get_dummies(bins)
    X=pd.concat([X, bin2], axis=1)
    
    oneHot=pd.get_dummies(X['Name'])
    X=pd.concat([X, oneHot], axis=1)
    oneHot=pd.get_dummies(X['SibSp'], prefix='SS')
    X=pd.concat([X, oneHot], axis=1)
    oneHot=pd.get_dummies(X['Parch'], prefix='PC')
    X=pd.concat([X, oneHot], axis=1)
    oneHot=pd.get_dummies(X['Sex'])
    X=pd.concat([X, oneHot], axis=1) 
    oneHot=pd.get_dummies(X['Pclass'], prefix='Class')
    X=pd.concat([X, oneHot], axis=1)
    oneHot=pd.get_dummies(X['Embarked'])
    X=pd.concat([X, oneHot], axis=1)
    
    return(X)

In [45]:
def interactVar(X):
    X['MrSibSpPclass']=X['Mr']*X['SibSp']*X['Pclass']
    X['MrsParchPclass']=X['Mrs']*X['Parch']*X['Pclass']
    X['MissPclass']=X['Miss']*X['Pclass']
    X['FarePclass']=X['Fare']*X['Pclass']
    X['FemalePclass']=X['female']*X['Pclass']
    return(X)

In [46]:
xTrain=xCleaner(xTrain)
xTest=xCleaner(xTest)

In [47]:
xTrain=interactVar(xTrain)
xTest=interactVar(xTest)

In [48]:
X=xTrain[['Miss', 'Mrs', 'toddler', 'adolescent', 'Class_3']]
y=yTrain
m=RandomForestClassifier(max_depth=1, n_estimators=500)
m.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=1, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [49]:
xTest1=xTest[['Miss', 'Mrs', 'toddler', 'adolescent', 'Class_3']]
yPred=m.predict(xTest1)
yPred.shape

(418,)

In [50]:
submit=pd.DataFrame({'Passengerid': xTest1.index, 'Survived': yPred})
submit.to_csv('rfTitanic.csv', index=False)
submit

,Passengerid,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,0
415,1307,0
416,1308,0
